Prediction helper (load saved model & vectorizer)

1. Imports and Setup

In [ ]:
import os
import re
import string
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Colab file utilities
try:
    from google.colab import files
except Exception:
    files = None

# Visualization helper
from wordcloud import WordCloud


print("1) Ensuring NLTK stopwords are available...")
nltk.download('stopwords', quiet=True)
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

2. Upload CSV File

In [ ]:
if files is None:
    raise EnvironmentError("This cell expects to run in Google Colab (google.colab.files not available).")

print("\n2) Upload your dataset CSV (e.g., spam.csv). Use the upload dialog that appears...")
uploaded = files.upload()
if not uploaded:
    raise FileNotFoundError("No file uploaded. Re-run the cell and upload the CSV file.")

# take first uploaded file
filename = next(iter(uploaded))
print("Uploaded file:", filename)

3. Read CSV with Encoding Fallback

In [ ]:
print("\n3) Reading CSV (trying utf-8, latin1, iso-8859-1)...")
encodings = ['utf-8', 'latin1', 'iso-8859-1']
df = None
for enc in encodings:
    try:
        df = pd.read_csv(filename, encoding=enc)
        print(f"Read with encoding: {enc}")
        break
    except Exception as e:
        print(f"Failed with {enc}: {type(e).__name__}")
if df is None:
    raise ValueError("Unable to read CSV with tried encodings. Please check file or rerun with a different file.")

# ------------------ Inspect & normalize columns ------------------
print("\n4) Inspecting columns:", list(df.columns))
# Common column names: 'v1','v2' or 'label','text' or 'class','message'
col_lower = [c.lower() for c in df.columns]

# find text column
text_col = None
for cand in ['text', 'message', 'body', 'msg', 'v2']:
    if cand in col_lower:
        text_col = df.columns[col_lower.index(cand)]
        break
# find label column
label_col = None
for cand in ['label', 'class', 'v1', 'type']:
    if cand in col_lower:
        label_col = df.columns[col_lower.index(cand)]
        break

if text_col is None or label_col is None:
    raise KeyError("Couldn't detect text and/or label columns automatically. Please ensure CSV has columns like ('v1','v2') or ('label','text'). Columns found: " + ", ".join(df.columns))

# rename for consistency
df = df.rename(columns={text_col: 'text', label_col: 'label'})
df = df[['text', 'label']].dropna().reset_index(drop=True)

print(f"Using text column: 'text' & label column: 'label' — sample rows: {min(3, len(df))}")
print(df.head(3))

# Normalize labels to lower-case strings
df['label'] = df['label'].astype(str).str.lower().str.strip()
# If labels are not ham/spam map: try common maps (1/0, spam/ham etc.)
if set(df['label'].unique()) - set(['spam','ham']) and set(df['label'].unique()) & set(['1','0','true','false']):
    # try numeric mapping
    df['label'] = df['label'].replace({'1': 'spam', '0': 'ham', 'true': 'spam', 'false': 'ham', 't': 'spam', 'f': 'ham'})
print("Label value counts:")
print(df['label'].value_counts())

4. Text Cleaning

In [ ]:
print("\n5) Cleaning text (lowercase, remove non-letters, remove stopwords, stemming)...")
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    # replace urls, emails, numbers with space
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)
    text = re.sub(r'\S+@\S+', ' ', text)
    text = re.sub(r'[^a-z\s]', ' ', text)  # keep letters and spaces only
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = text.split()
    tokens = [stemmer.stem(tok) for tok in tokens if tok not in stop_words]
    return " ".join(tokens)

# Apply cleaning with progress if large
df['clean_text'] = df['text'].apply(clean_text)
print("Example original -> cleaned:")
for i in range(min(2, len(df))):
    print("ORIG:", df['text'].iloc[i])
    print("CLEAN:", df['clean_text'].iloc[i])
    print("---")

# Remove rows with empty cleaned text
before = len(df)
df = df[df['clean_text'].str.strip() != ""].reset_index(drop=True)
after = len(df)
if before != after:
    print(f"Removed {before-after} rows with empty cleaned text.")

5. Feature Extraction (TF-IDF)

In [ ]:
print("\n6) TF-IDF vectorization (max_features=5000)...")
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_text']).toarray()
# Map labels to binary
label_map = {'ham': 0, 'spam': 1}
if not set(df['label'].unique()).issubset(set(label_map.keys())):
    # attempt to map anything else: take most frequent as ham and other as spam if unclear
    uniq = list(df['label'].unique())
    print("Unrecognized label values detected:", uniq)
    # try numeric mapping if possible
    df['label'] = df['label'].apply(lambda x: 'spam' if x in ['1','yes','y','spam'] else ('ham' if x in ['0','no','n','ham'] else x))
    # final fallback: map top freq to ham, rest to spam
    if not set(df['label'].unique()).issubset(set(label_map.keys())):
        most_common = df['label'].value_counts().idxmax()
        df['label'] = df['label'].apply(lambda x: 'ham' if x==most_common else 'spam')
y = df['label'].map(label_map).values

print("TF-IDF shape:", X.shape)
print("Label distribution (binary):")
print(pd.Series(y).value_counts())

6. Data Splitting

In [ ]:
print("\n7) Splitting data (train/test 80/20)...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

7. Model Training

In [ ]:
print("\n8) Training models: MultinomialNB, SVM (linear), RandomForest")
models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(kernel='linear', probability=True),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

trained_models = {}
scores = {}
for name, model in models.items():
    try:
        print(f"\nTraining {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        scores[name] = acc
        trained_models[name] = model
        print(f"{name} Accuracy: {acc:.4f}")
        print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(4,3))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ham','Spam'], yticklabels=['Ham','Spam'])
        plt.title(f"{name} Confusion Matrix")
        plt.show()
    except Exception as e:
        print(f"Error training {name}: {e}")

8. Best Model Selection

In [ ]:
if not scores:
    raise RuntimeError("No models trained successfully.")
best_name = max(scores, key=scores.get)
best_model = trained_models[best_name]
print(f"\n9) Best model based on accuracy: {best_name} (Accuracy={scores[best_name]:.4f})")

9. Word Clouds

In [ ]:
print("\n10) Generating word clouds for spam and ham (may take a moment)...")
try:
    spam_text = " ".join(df[df['label']=='spam']['clean_text'])
    ham_text = " ".join(df[df['label']=='ham']['clean_text'])
    def show_wc(text, title):
        if not text.strip():
            print("No text for", title)
            return
        wc = WordCloud(width=800, height=400, background_color='white').generate(text)
        plt.figure(figsize=(10,4))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis('off')
        plt.title(title)
        plt.show()
    show_wc(spam_text, "Spam Word Cloud")
    show_wc(ham_text, "Ham Word Cloud")
except Exception as e:
    print("Word cloud generation failed:", e)

10. Save Model and Vectorizer

In [ ]:
model_filename = 'best_spam_model.joblib'
vec_filename = 'tfidf_vectorizer.joblib'
print(f"\n11) Saving best model ({best_name}) to {model_filename} and vectorizer to {vec_filename} ...")
joblib.dump(best_model, model_filename)
joblib.dump(tfidf, vec_filename)

# Download files (Colab)
if files is not None:
    try:
        files.download(model_filename)
        files.download(vec_filename)
    except Exception as e:
        print("Automatic download failed (this sometimes happens). Files are saved in the Colab working directory. Use the file browser to download manually. Error:", e)

11. Prediction Function and Testing

In [ ]:
print("\n12) Defining predict_email(...) function and testing with samples.")

def predict_email(email_text, model_path=model_filename, vec_path=vec_filename):
    # load model & vectorizer
    m = joblib.load(model_path)
    v = joblib.load(vec_path)
    cleaned = clean_text(email_text)
    vec = v.transform([cleaned]).toarray()
    pred = m.predict(vec)[0]
    prob = None
    try:
        if hasattr(m, "predict_proba"):
            p = m.predict_proba(vec)[0]
            prob = {'ham_prob': f"{p[0]*100:.2f}%", 'spam_prob': f"{p[1]*100:.2f}%"}
    except Exception:
        prob = None
    return {'prediction': 'spam' if pred==1 else 'ham', 'probabilities': prob, 'cleaned_text': cleaned}

# Test samples
sample_ham = "Hi there, just checking in to see if you're free for lunch tomorrow?"
sample_spam = "CONGRATULATIONS!! You've won a $1000 Walmart gift card. Click here to claim!"

print("Sample ham ->", predict_email(sample_ham))
print("Sample spam ->", predict_email(sample_spam))

